Code used to create tables in the PIC paper

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import pymysql

In [35]:
# Connect to PIC
con = pymysql.connect(host='localhost',
                             user='root',
                             password='13163232289zx',
                             db='pmimic',
                             cursorclass=pymysql.cursors.DictCursor)
cur=con.cursor()

How many patients who had multiple hospital admissions

In [9]:
query = \
"""
WITH chartobs AS (
SELECT subject_id, count(subject_id) as obs
FROM admissions
GROUP BY subject_id)
SELECT count(obs)
FROM chartobs
where obs>1;
"""
query_output = pd.read_sql_query(query,con)
print(query_output.head())

   count(obs)
0         468


These are the details that we would like to include in Table 2, grouped by first careunit:
  Patients, no. (% of total patients)
  Hospital admissions, no. (% of total admissions)
  Distinct ICU stays, no. (% of total unit stays)
  Age, yrs, mean ± SD
  Gender, male, percent of unit stays
  ICU length of stay, median days (IQR)
  Hospital length of stay, median days (IQR)
  ICU mortality, percent of unit stays
  Hospital mortality, percent of unit stays

Extract and review the data

In [13]:
# Join admissions, icustays, and patients tables

query = \
"""
WITH population as (
SELECT a.subject_id, a.hadm_id, i.icustay_id, 
    a.admittime as hosp_admittime, a.dischtime as hosp_dischtime, 
    i.first_careunit, 
    p.dob, p.dod, i.intime as icu_intime, i.outtime as icu_outtime, 
    round(timestampdiff(SECOND,i.intime,i.outtime)/60/60/24,4) as icu_los,
    round(timestampdiff(SECOND,a.admittime,a.dischtime)/60/60/24,4)   as hosp_los, 
    p.gender, 
    round(timestampdiff(SECOND,p.dob,a.admittime)/60/60/24/365,4) as age_hosp_in,
    round(timestampdiff(SECOND,p.dob,i.intime)/60/60/24/365,4) as age_icu_in,
    hospital_expire_flag,
    CASE WHEN p.dod IS NOT NULL 
        AND p.dod >= i.intime
        AND p.dod <= i.outtime THEN 1 
        ELSE 0 END AS icu_expire_flag
FROM admissions a
INNER JOIN icustays i
ON a.hadm_id = i.hadm_id
INNER JOIN patients p
ON a.subject_id = p.subject_id
ORDER BY a.subject_id, i.intime
)
SELECT *
FROM population;
"""

query_output = pd.read_sql_query(query,con)
print(query_output.head())

   subject_id  hadm_id  icustay_id      hosp_admittime      hosp_dischtime  \
0           2   100183      200715 2081-08-18 09:31:33 2081-09-28 10:06:00   
1           3   100490      201022 2108-06-27 17:07:48 2108-07-04 10:17:00   
2           4   102191      202747 2081-09-25 10:28:15 2081-10-08 10:46:00   
3           5   100548      201080 2075-08-19 15:26:27 2075-08-31 11:21:00   
4           6   100199      200731 2086-08-31 14:17:23 2086-09-11 13:39:00   

  first_careunit                 dob                 dod          icu_intime  \
0    General ICU 2068-11-30 00:00:00 2081-09-28 10:06:00 2081-08-18 09:31:32   
1    General ICU 2100-02-18 00:00:00                 NaT 2108-06-27 17:07:48   
2    General ICU 2069-11-08 10:30:00                 NaT 2081-09-25 21:29:52   
3    General ICU 2059-10-08 00:00:00 2075-08-31 11:21:00 2075-08-19 15:26:26   
4    General ICU 2079-05-13 00:00:00                 NaT 2086-08-31 14:17:22   

          icu_outtime  icu_los  hosp_los gender  a

Distinct patients, no. (% of total patients)

In [21]:
print('\nTotal patients: {}'\
    .format(len(query_output.subject_id.unique())))

print('\nNumber of patients by first careunit:\n')
print(query_output[['first_careunit','subject_id']] \
                    .drop_duplicates(['subject_id']) \
                    .groupby('first_careunit').count().T)
    
print('\nProportion of total hospital admissions:\n')
print(query_output[['first_careunit','subject_id']] \
                    .drop_duplicates(['subject_id']) \
                    .groupby('first_careunit') \
                    .count().T/len(query_output.subject_id.unique())*100)


Total patients: 12881

Number of patients by first careunit:

first_careunit  CICU  General ICU  NICU  PICU  SICU
subject_id      2583         2642  3137  1953  2566

Proportion of total hospital admissions:

first_careunit       CICU  General ICU       NICU       PICU       SICU
subject_id      20.052791     20.51083  24.353699  15.161866  19.920814


Hospital admissions, no. (% of total admissions)

In [22]:
print('\nTotal hospital admissions: {}'\
    .format(len(query_output.hadm_id.unique())))

print('\nNumber of hospital admissions by first careunit:\n')
print(query_output[['first_careunit','hadm_id']] \
                    .drop_duplicates(['hadm_id']) \
                    .groupby('first_careunit').count().T)
    
print('\nProportion of total hospital admissions:\n')
print(query_output[['first_careunit','hadm_id']] \
                    .drop_duplicates(['hadm_id']) \
                    .groupby('first_careunit') \
                    .count().T/len(query_output.hadm_id.unique())*100)


Total hospital admissions: 13449

Number of hospital admissions by first careunit:

first_careunit  CICU  General ICU  NICU  PICU  SICU
hadm_id         2638         2725  3205  2084  2797

Proportion of total hospital admissions:

first_careunit       CICU  General ICU       NICU       PICU       SICU
hadm_id         19.614841    20.261729  23.830768  15.495576  20.797085


Distinct ICU stays, no. (% of total unit stays)

In [23]:
print('\nTotal ICU stays: {}'\
    .format(len(query_output.icustay_id.unique())))

print('\nNumber of ICU stays by careunit:\n')
print(query_output[['first_careunit','icustay_id']] \
          .groupby('first_careunit').count().T)

print('\nProportion of total ICU stays:\n')
print(query_output[['first_careunit','icustay_id']] \
          .groupby('first_careunit') \
          .count().T/len(query_output.icustay_id.unique())*100)


Total ICU stays: 13941

Number of ICU stays by careunit:

first_careunit  CICU  General ICU  NICU  PICU  SICU
icustay_id      2803         2788  3282  2166  2902

Proportion of total ICU stays:

first_careunit       CICU  General ICU      NICU       PICU       SICU
icustay_id      20.106162    19.998565  23.54207  15.536906  20.816297


Age, years, mean (IQR)

In [26]:
print('Mean age, years: {} '.format(query_output.age_icu_in.mean()))
print('Lower quartile age, years: {} '.format(query_output.age_icu_in.quantile(0.25)))
print('Upper quartile age, years: {} \n '.format(query_output.age_icu_in.quantile(0.75)))

print('Mean age by careunit, years:\n ')
print(query_output[['first_careunit','age_icu_in']] \
      .groupby('first_careunit').mean().T)

print('\nLower quartile by careunit, years:\n ')
print(query_output[['first_careunit','age_icu_in']] \
      .groupby('first_careunit').quantile(0.25).T)

print('\nUpper quartile by careunit, years:\n ')
print(query_output[['first_careunit','age_icu_in']] \
      .groupby('first_careunit').quantile(0.75).T)

Mean age, years: 2.480802532099554 
Lower quartile age, years: 0.1194 
Upper quartile age, years: 3.3397 
 
Mean age by careunit, years:
 
first_careunit      CICU  General ICU      NICU      PICU      SICU
age_icu_in      2.462498     3.775664  0.040844  3.622367  3.161902

Lower quartile by careunit, years:
 
first_careunit     CICU  General ICU   NICU    PICU     SICU
0.25                                                        
age_icu_in      0.45265      0.41235  0.002  0.5123  0.37945

Upper quartile by careunit, years:
 
first_careunit     CICU  General ICU      NICU      PICU      SICU
0.75                                                              
age_icu_in      3.13975         6.27  0.049475  5.704025  4.869075


Gender, male, percent of unit stays

In [27]:
print('Gender:\n')
print(query_output.groupby('gender').gender.count())
print(query_output.groupby('gender').gender.count() \
     /query_output.gender.count()*100)

print('Gender by careunit:\n')
print(query_output.groupby(['first_careunit','gender']).gender.count())

print('\nProportion by unit:\n')
print(query_output.groupby(['first_careunit','gender']) \
    .gender.count()/query_output.groupby(['first_careunit']).gender.count()*100)

Gender:

gender
F    5924
M    8017
Name: gender, dtype: int64
gender
F    42.493365
M    57.506635
Name: gender, dtype: float64
Gender by careunit:

first_careunit  gender
CICU            F         1412
                M         1391
General ICU     F         1076
                M         1712
NICU            F         1298
                M         1984
PICU            F          916
                M         1250
SICU            F         1222
                M         1680
Name: gender, dtype: int64

Proportion by unit:

first_careunit  gender
CICU            F         50.374599
                M         49.625401
General ICU     F         38.593974
                M         61.406026
NICU            F         39.549055
                M         60.450945
PICU            F         42.289935
                M         57.710065
SICU            F         42.108890
                M         57.891110
Name: gender, dtype: float64


ICU length of stay, mean days (IQR)

In [28]:
print('Mean ICU length of stay, days: {}'.format(query_output.icu_los.mean()))
print('Lower quartile ICU length of stay, days: {}' \
    .format(query_output.icu_los.quantile(0.25)))
print('Upper quartile ICU length of stay, days: {}\n' \
    .format(query_output.icu_los.quantile(0.75)))

print('Mean length of ICU stay by careunit, days:\n ')
print(query_output[['first_careunit','icu_los']] \
      .groupby('first_careunit').mean().T)

print('\nLower quartile length of ICU stay, days:\n ')
print(query_output[['first_careunit','icu_los']] \
      .groupby('first_careunit').quantile(0.25).T)

print('\nUpper quartile length of ICU stay, days:\n ')
print(query_output[['first_careunit','icu_los']] \
      .groupby('first_careunit').quantile(0.75).T)

Mean ICU length of stay, days: 9.30284736389066
Lower quartile ICU length of stay, days: 0.9217
Upper quartile ICU length of stay, days: 9.1691

Mean length of ICU stay by careunit, days:
 
first_careunit      CICU  General ICU       NICU      PICU      SICU
icu_los         3.926365     7.292125  21.557103  9.710644  2.264398

Lower quartile length of ICU stay, days:
 
first_careunit    CICU  General ICU    NICU     PICU    SICU
0.25                                                        
icu_los         0.9172     0.886975  2.5477  1.98555  0.7767

Upper quartile length of ICU stay, days:
 
first_careunit    CICU  General ICU      NICU       PICU      SICU
0.75                                                              
icu_los         3.9487     8.911775  32.82785  11.136225  1.625525


Hospital length of stay, mean days (IQR)

In [30]:
print('Mean length of hospital stay, days: {}' \
    .format(query_output.drop_duplicates(['hadm_id']).hosp_los.mean()))
print('Lower quartile length of hospital stay, days: {}' \
    .format(query_output.drop_duplicates(['hadm_id']).hosp_los.quantile(0.25)))
print('Upper quartile length of hospital stay, days: {}\n' \
    .format(query_output.drop_duplicates(['hadm_id']).hosp_los.quantile(0.75)))

print('Mean length of hospital stay, days:\n ')
print(query_output.drop_duplicates(['hadm_id']) \
      [['first_careunit','hosp_los']] \
      .groupby('first_careunit').mean().T)

print('\nLower quartile length of hospital stay, days:\n ')
print(query_output.drop_duplicates(['hadm_id']) \
      [['first_careunit','hosp_los']] \
      .groupby('first_careunit').quantile(0.25).T)

print('\nUpper quartile length of hospital stay, days:\n ')
print(query_output.drop_duplicates(['hadm_id']) \
      [['first_careunit','hosp_los']] \
      .groupby('first_careunit').quantile(0.75).T)

Mean length of hospital stay, days: 17.589063320692958
Lower quartile length of hospital stay, days: 7.0254
Upper quartile length of hospital stay, days: 20.9715

Mean length of hospital stay, days:
 
first_careunit      CICU  General ICU       NICU       PICU       SICU
hosp_los        16.56631    12.777065  27.078793  14.520918  14.653829

Lower quartile length of hospital stay, days:
 
first_careunit      CICU  General ICU    NICU    PICU   SICU
0.25                                                        
hosp_los        9.031225       3.8202  9.6134  4.5536  7.023

Upper quartile length of hospital stay, days:
 
first_careunit     CICU  General ICU    NICU      PICU     SICU
0.75                                                           
hosp_los        19.1531      16.2091  37.839  16.71495  18.7365


ICU mortality, percent of unit stays

In [31]:
print('ICU mortality, number:\n')
print(query_output \
    .groupby(['icu_expire_flag']) \
    .icu_expire_flag.count())

print('\nICU mortality, %:\n')
print(query_output.groupby(['icu_expire_flag']) \
    .icu_expire_flag.count() / query_output.icu_expire_flag.count()*100)

print('\nICU mortality by careunit:\n')
print(query_output \
    .groupby(['first_careunit','icu_expire_flag']) \
    .icu_expire_flag.count())

print('\nProportion by unit:\n')
print(query_output \
    .groupby(['first_careunit','icu_expire_flag']) \
    .icu_expire_flag.count()/query_output \
    .groupby(['first_careunit']).icu_expire_flag.count()*100)

ICU mortality, number:

icu_expire_flag
0    12986
1      955
Name: icu_expire_flag, dtype: int64

ICU mortality, %:

icu_expire_flag
0    93.149702
1     6.850298
Name: icu_expire_flag, dtype: float64

ICU mortality by careunit:

first_careunit  icu_expire_flag
CICU            0                  2755
                1                    48
General ICU     0                  2374
                1                   414
NICU            0                  3046
                1                   236
PICU            0                  1966
                1                   200
SICU            0                  2845
                1                    57
Name: icu_expire_flag, dtype: int64

Proportion by unit:

first_careunit  icu_expire_flag
CICU            0                  98.287549
                1                   1.712451
General ICU     0                  85.150646
                1                  14.849354
NICU            0                  92.809263
                1     

Hospital mortality, percent of hospital stays

In [33]:
print('Hospital mortality, number:\n')
print(query_output.drop_duplicates(['hadm_id']) \
    .groupby(['hospital_expire_flag']) \
    .hospital_expire_flag.count())

print('\nHospital mortality, %:\n')
print(query_output.drop_duplicates(['hadm_id']) \
    .groupby(['hospital_expire_flag']) \
    .hospital_expire_flag.count() \
    / query_output.drop_duplicates(['hadm_id']).hospital_expire_flag.count()*100)

print('\nHospital mortality:\n')
print(query_output.drop_duplicates(['hadm_id']) \
    .groupby(['first_careunit','hospital_expire_flag']) \
    .hospital_expire_flag.count())

print('\nProportion by unit:\n')
print(query_output.drop_duplicates(['hadm_id']) \
    .groupby(['first_careunit','hospital_expire_flag']) \
    .hospital_expire_flag.count()/query_output.drop_duplicates(['hadm_id']) \
    .groupby(['first_careunit']).hospital_expire_flag.count()*100)

Hospital mortality, number:

hospital_expire_flag
0    12478
1      971
Name: hospital_expire_flag, dtype: int64

Hospital mortality, %:

hospital_expire_flag
0    92.780132
1     7.219868
Name: hospital_expire_flag, dtype: float64

Hospital mortality:

first_careunit  hospital_expire_flag
CICU            0                       2585
                1                         53
General ICU     0                       2308
                1                        417
NICU            0                       2966
                1                        239
PICU            0                       1879
                1                        205
SICU            0                       2740
                1                         57
Name: hospital_expire_flag, dtype: int64

Proportion by unit:

first_careunit  hospital_expire_flag
CICU            0                       97.990902
                1                        2.009098
General ICU     0                       84.697248
         

Table 3: Distribution of primary diagnosis by care unit for patients in the PIC database

In [59]:
query = \
"""

SELECT a.subject_id,a.hadm_id,i.icustay_id,i.first_careunit,icd.icd10_code_cn,
    lower(LEFT(icd.icd10_code_cn,1)) AS icd_first1, SUBSTRING(icd.icd10_code_cn,2,1) AS icd_first2,
    SUBSTRING(icd.icd10_code_cn,4,1) AS icd_first4
    FROM admissions a
    INNER JOIN icustays i
    ON a.hadm_id = i.hadm_id
    INNER JOIN patients p
    ON a.subject_id = p.subject_id
    INNER JOIN diagnoses_icd icd
    ON a.hadm_id=icd.hadm_id
"""
query_output = pd.read_sql_query(query,con)
print(query_output.head())

   subject_id  hadm_id  icustay_id first_careunit icd10_code_cn icd_first1  \
0           2   100183      200715    General ICU       Z94.801          z   
1           3   100490      201022    General ICU       R40.201          r   
2           4   102191      202747    General ICU       K56.200          k   
3           5   100548      201080    General ICU       C95.901          c   
4           6   100199      200731    General ICU        N19.02          n   

  icd_first2 icd_first4  
0          9          .  
1          4          .  
2          5          .  
3          9          .  
4          1          .  


In [51]:
#process icd-10 codes category
def icd_category(icd1,icd2,icd4):
    icd2=int(icd2)
    if icd1=='a' or icd1=='b':
        icd_category='A00-B99'
    if icd1=='c':
        icd_category='C00-D48'
    if icd1=='d' and icd2>=0 and icd2<=4:
        icd_category='C00-D48'
    if icd1=='d' and icd2>=5 and icd2<=8:
        icd_category='D50-D89'
    if icd1=='e':
        icd_category='E00-E90'
    if icd1=='f':
        icd_category='F00-F90'
    if icd1=='g':
        icd_category='G00-G90'
    if icd1=='h' and icd2>=0 and icd2<=5:
        icd_category='H00-H59'
    if icd1=='h' and icd2>=6 and icd2<=9:
        icd_category='H60-H95'
    if icd1=='i':
        icd_category='I00-I99'
    if icd1=='j':
        icd_category='J00-J99'
    if icd1=='k':
        icd_category='K00-K93'
    if icd1=='l':
        icd_category='L00-L99'
    if icd1=='m' and icd4=='.':
        icd_category='M00-M99'
    if icd1=='n':
        icd_category='N00-N99'
    if icd1=='o':
        icd_category='O00-O99'
    if icd1=='p':
        icd_category='P00-P96'
    if icd1=='q':
        icd_category='Q00-Q99'
    if icd1=='r':
        icd_category='R00-R99'
    if icd1=='s' or icd1=='t':
        icd_category='S00-T98'
    if icd1=='v' or icd1=='w' or icd1=='x' or icd1=='y':
        icd_category='V01-Y98'
    if icd1=='z':
        icd_category='Z00-Z99'
    if icd1=='m' and icd4!='.':
        icd_category='M'
    return icd_category
    
query_output['icd_category']=list(map(lambda icd1,icd2,icd4:icd_category(icd1,icd2,icd4),query_output['icd_first1'],query_output['icd_first2'],query_output['icd_first4']))

Primary ICD-10 codes by careunit

In [58]:
print('Primary ICD diagnoses by ICU stay:\n')
a=query_output.drop_duplicates(['hadm_id']).loc[query_output.first_careunit=='CICU']['icd_category'].value_counts()
b=query_output.drop_duplicates(['hadm_id']).loc[query_output.first_careunit=='General ICU']['icd_category'].value_counts()
c=query_output.drop_duplicates(['hadm_id']).loc[query_output.first_careunit=='NICU']['icd_category'].value_counts()
d=query_output.drop_duplicates(['hadm_id']).loc[query_output.first_careunit=='PICU']['icd_category'].value_counts()
e=query_output.drop_duplicates(['hadm_id']).loc[query_output.first_careunit=='SICU']['icd_category'].value_counts()
f=query_output.drop_duplicates(['hadm_id'])['icd_category'].value_counts()
df_num=pd.concat([a,b,c,d,e,f],axis=1,sort=False)
df_num.fillna(value=0, inplace=True)
df_num.columns = ['CICU', 'General ICU','NICU','PICU','SICU','Total']
# Append a totals row
print(df_num.sort_index())

print('\nProportion by careunit:\n')
a=(a/query_output.drop_duplicates(['hadm_id']).loc[query_output.first_careunit=='CICU']['icd_category'].count())*100
b=(b/query_output.drop_duplicates(['hadm_id']).loc[query_output.first_careunit=='General ICU']['icd_category'].count())*100
c=(c/query_output.drop_duplicates(['hadm_id']).loc[query_output.first_careunit=='NICU']['icd_category'].count())*100
d=(d/query_output.drop_duplicates(['hadm_id']).loc[query_output.first_careunit=='PICU']['icd_category'].count())*100
e=(e/query_output.drop_duplicates(['hadm_id']).loc[query_output.first_careunit=='SICU']['icd_category'].count())*100
f=(f/query_output.drop_duplicates(['hadm_id'])['icd_category'].count())*100
df_percent=pd.concat([a,b,c,d,e,f],axis=1,sort=False)
df_percent.fillna(value=0, inplace=True)
df_percent.columns = ['CICU', 'General ICU','NICU','PICU','SICU','Total']
# Append a totals row
print(df_percent.sort_index())

Primary ICD diagnoses by ICU stay:

           CICU  General ICU    NICU   PICU   SICU  Total
A00-B99     4.0        117.0    44.0   91.0   12.0    268
C00-D48    26.0        198.0     3.0   49.0  230.0    506
D50-D89     0.0        167.0    11.0   95.0   10.0    283
E00-E90     4.0         64.0    37.0   57.0   22.0    184
F00-F90     0.0         58.0     1.0    2.0    2.0     63
G00-G90     0.0        201.0    38.0  274.0  246.0    759
H00-H59     0.0          0.0     9.0    2.0    8.0     19
H60-H95     0.0          0.0     0.0    1.0    3.0      4
I00-I99   314.0        110.0    78.0  153.0  110.0    765
J00-J99   189.0        430.0   173.0  518.0   65.0   1375
K00-K93    56.0        184.0   184.0  102.0  483.0   1009
L00-L99     0.0          7.0     4.0    7.0   19.0     37
M           5.0         31.0     7.0   27.0  150.0    220
M00-M99     0.0         43.0    14.0   11.0    7.0     75
N00-N99     0.0        388.0     8.0   13.0   12.0    421
O00-O99     0.0          0.0     4.0

Table 5 Amount of complete data grouped by table

In [65]:
query1 = \
"""
SELECT distinct hadm_id from chartevents
"""
query_output1 = pd.read_sql_query(query1,con)
print('CHARTEVENTS: {}' \
    .format((len(query_output1)/13449)*100))
query2 = \
"""
SELECT distinct hadm_id from diagnoses_icd
"""
query_output2 = pd.read_sql_query(query2,con)
print('DIAGNOSES_ICD: {}' \
    .format((len(query_output2)/13449)*100))
query3 = \
"""
SELECT distinct hadm_id from emr_symptoms
"""
query_output3 = pd.read_sql_query(query3,con)
print('EMR_SYMPTOMS: {}' \
    .format((len(query_output3)/13449)*100))
query4 = \
"""
SELECT distinct hadm_id from inputevents
"""
query_output4 = pd.read_sql_query(query4,con)
print('INPUTEVENTS: {}' \
    .format((len(query_output4)/13449)*100))
query5 = \
"""
SELECT distinct hadm_id from labevents
"""
query_output5 = pd.read_sql_query(query5,con)
print('LABEVENTS: {}' \
    .format((len(query_output5)/13449)*100))
query6 = \
"""
SELECT distinct hadm_id from microbiologyevents
"""
query_output6 = pd.read_sql_query(query6,con)
print('MICROBIOLOGYEVENTS: {}' \
    .format((len(query_output6)/13449)*100))
query7 = \
"""
SELECT distinct hadm_id from or_exam_reports
"""
query_output7 = pd.read_sql_query(query7,con)
print('OR_EXAM_REPORTS: {}' \
    .format((len(query_output7)/13449)*100))
query8 = \
"""
SELECT distinct hadm_id from outputevents
"""
query_output8 = pd.read_sql_query(query8,con)
print('OUTPUTEVENTS: {}' \
    .format((len(query_output8)/13449)*100))
query9 = \
"""
SELECT distinct hadm_id from prescriptions
"""
query_output9 = pd.read_sql_query(query9,con)
print('PRESCRIPTIONS: {}' \
    .format((len(query_output9)/13449)*100))
query10 = \
"""
SELECT distinct hadm_id from surgery_vital_signs
"""
query_output10 = pd.read_sql_query(query10,con)
print('SURGERY_VITAL_SIGNS: {}' \
    .format((len(query_output10)/13449)*100))

CHARTEVENTS: 79.70852851513123
DIAGNOSES_ICD: 99.37541824670978
EMR_SYMPTOMS: 6.654769871365901
INPUTEVENTS: 41.095992267083055
LABEVENTS: 93.85084392891665
MICROBIOLOGYEVENTS: 89.69440107071158
OR_EXAM_REPORTS: 91.91017919547922
OUTPUTEVENTS: 11.666294891813518
PRESCRIPTIONS: 51.48338166406424
SURGERY_VITAL_SIGNS: 47.71358465313406
